![title](../NAG_logo.png)
# Regression

## Fitting linear regression models

The `correg.linregm_fit` function fits a linear (multiple) regression model for a set of independent variables $\mathbf{x}_1,\ldots,\mathbf{x}_m$ and dependent observations $\mathbf{y}=(y_1,\ldots,y_n)$.

For illustrative purposes the exercises use synthetically generated data.  Here the distribution of $\mathbf{y}$ given $\mathbf{x}_1,\ldots,\mathbf{x}_m$ is normal.

Use the code below to generate synthetic observations.  Find the example for `correg.linregm_fit`.  Use it to help you fit a linear regression model to the synthetic observations.  Print out standard errors of the parameter estimates.

How does the standard error of parameter estimates change with the number of observations, $n$?

*Note*: you can locate the source code for all the examples by running,

`python -m naginterfaces.library.examples --locate` 

from the command-line.

In [1]:
# import modules
import numpy as np
from naginterfaces.library import rand, correg
import reg_fun
# make sure modules are reloaded if we are make changes
import importlib
importlib.reload(reg_fun)

# number of indepdent variables
m = 5
# number of observations
n = 2e1
# initialize RNG
# statecomm = rand.init_nonrepeat(genid=3)          # either with a random seed
statecomm = rand.init_repeat(genid=3, seed=[32958]) # or with a fixed seed
    
# The independent variables:
x = reg_fun.gen_multivar_x(m, n, statecomm)
# The observations:
y = reg_fun.gen_obs(x, statecomm)
# The independent variables to include:
isx = np.ones(m,int)

fit_lin = correg.linregm_fit(x, isx, y)

print('Term' + ' '*9 + 'Estimate' + ' '*3 + 'Standard Error')
for i, b_i in enumerate(fit_lin.b):
    print('Variable:' + ' '*2 + '{:d} {:.3e}   {:.3e}'.format(
        i, b_i, fit_lin.se[i]
    ))


Term         Estimate   Standard Error
Variable:  0 -7.699e-01   1.113e+00
Variable:  1 5.370e-01   6.534e-01
Variable:  2 1.613e+00   5.218e-01
Variable:  3 1.493e+00   5.546e-01
Variable:  4 2.115e-01   4.952e-01
Variable:  5 2.516e+00   5.858e-01


## Fitting Generalized Linear Models (GLMs)

The `correg.glm_normal` function fits a GLM with normally distributed errors, again for a set of independent variables $\mathbf{x}_1,\ldots,\mathbf{x}_m$ and dependent observations $\mathbf{y}=(y_1,\ldots,y_n)$.

Identify a value for the `link` argument that results in the same linear regression model as the previous exercise.  Use `correg.glm_normal` to fit the same data as in the previous exercise.  Check that you get the same parameter estimates.

In [2]:
from naginterfaces.base import utils

# Other parameters for the glm_normal function:
link = 'I'
iom = utils.FileObjManager()
tol = 5.e-5
eps = 1.e-6

fit_glm = correg.glm_normal(x, isx, y, link=link, tol=tol, eps=eps, io_manager=iom)

print('Term' + ' '*9 + 'Estimate' + ' '*3 + 'Standard Error')
for i, b_i in enumerate(fit_glm.b):
    print('Variable:' + ' '*2 + '{:d} {:.3e}   {:.3e}'.format(
        i, b_i, fit_glm.se[i]
    ))

Term         Estimate   Standard Error
Variable:  0 -7.699e-01   1.113e+00
Variable:  1 5.370e-01   6.534e-01
Variable:  2 1.613e+00   5.218e-01
Variable:  3 1.493e+00   5.546e-01
Variable:  4 2.115e-01   4.952e-01
Variable:  5 2.516e+00   5.858e-01


## GLM prediction intervals

Generate some more synthetic data.  Use the `correg.glm_predict` function to make predictions for these new observations.  Calculate the Root Mean Squared Error (RMSE) of the predictions.  Compare with the standard errors output by `correg.glm_predict`.

In [3]:
# generate test dataset
x = reg_fun.gen_multivar_x(m, n, statecomm)
y = reg_fun.gen_obs(x, statecomm)

# Parameters for the glm_predict function:
errfn = 'N'
bhat = fit_glm.b
cov_bhat = fit_glm.cov
vfobs = True
s = fit_glm.s

(_, _, yhat, se_yhat) = correg.glm_predict(errfn, x, isx, bhat, cov_bhat, vfobs, link=link, s=s)
mse = np.sqrt( np.mean( (y - yhat) ** 2 ) )
print('Root Mean Squared Prediction Error')
print( mse )
print('Min, Mean and Max Standard Error in Predictions')
print( [np.min(se_yhat), np.mean(se_yhat), np.max(se_yhat)] )

vfobs = False
(_, _, yhat, se_yhat) = correg.glm_predict(errfn, x, isx, bhat, cov_bhat, vfobs, link=link, s=s)
mse = np.sqrt( np.mean( (y - yhat) ** 2 ) )
print('Min, Mean and Max Standard Error in Mean Estimates')
print( [np.min(se_yhat), np.mean(se_yhat), np.max(se_yhat)] )

Root Mean Squared Prediction Error
4.065502980243891
Min, Mean and Max Standard Error in Predictions
[3.7125733579824165, 4.137046560763536, 4.954932672505831]
Min, Mean and Max Standard Error in Mean Estimates
[1.360364609883751, 2.2193315460760785, 3.5522878152662556]


## Identifying outliers

Suppose that there is an error in one of the entries in your dataset, e.g., someone added a couple of extra zeros to an observation by mistake when recording the data.

Use the code below to explore what effect this has on fitting.  How do the regression coefficents change?  Extract the residuals and the leverages from the auxiliary information / output of the fitting.  Which of these can be used to identify the outlier / erroneous observation?

*Hint:* to ease printing of multiple output columns copy the output you are interested in to a `pandas` DataFrame.

In [4]:
import pandas as pd

xc = x.copy()  # copy x
xc[2,3] = 100.0 * x[2,3] # make 1 data point an outlier (3rd observation, 4th variable)

fit_glm = correg.glm_normal(xc, isx, y, link=link, tol=tol, eps=eps, io_manager=iom)

print('Term' + ' '*9 + 'Estimate' + ' '*3 + 'Standard Error')
for i, b_i in enumerate(fit_glm.b):
    print('Variable:' + ' '*2 + '{:d} {:.3e}   {:.3e}'.format(
        i, b_i, fit_glm.se[i]
    ))
    
pd.DataFrame(data=fit_glm.v[:,[4,5]], columns=['Residuals', 'Leverages'])

Term         Estimate   Standard Error
Variable:  0 -3.126e-01   1.419e+00
Variable:  1 8.155e-01   7.058e-01
Variable:  2 6.279e-01   7.242e-01
Variable:  3 8.652e-01   4.599e-01
Variable:  4 -9.679e-03   3.187e-02
Variable:  5 2.261e+00   4.383e-01


,Residuals,Leverages
0,-3.872923,0.429539
1,-2.025559,0.156837
2,-0.177670,0.998101
3,-1.403954,0.289301
4,0.440206,0.362906
5,2.971625,0.214027
6,0.000000,0.446335
7,-4.551681,0.311292
8,2.572436,0.177259
9,0.000000,0.214946
